In [6]:
!pip install atlassian-python-api

In [7]:
from atlassian import Confluence
import os
from bs4 import BeautifulSoup
import pandas as pd


In [8]:
user = "mortmafaka@gmail.com"
api_key = "wvDJF36QIKtIsoDLXln186D3"
server = "https://nmtechnologies.atlassian.net/wiki/spaces/NSJ"



In [9]:
confluence = Confluence(url=server, username=user, password=api_key)


In [10]:
from atlassian import Confluence
import os
from bs4 import BeautifulSoup
import pandas as pd

# Укажите ваши учетные данные и URL сервера Confluence
user = "mortmafaka@gmail.com"
api_key = "wvDJF36QIKtIsoDLXln186D3"
server = "https://nmtechnologies.atlassian.net"

# Создайте экземпляр класса Confluence для взаимодействия с сервером
confluence = Confluence(url=server, username=user, password=api_key)

# Получите список всех страниц из указанного пространства (space)
all_pages = confluence.get_all_pages_from_space(space="NSJ")

# Создайте пустой список, в который будут сохраняться данные каждой страницы
all_data = []

# Переберите список всех страниц и получите данные каждой страницы
for page in all_pages:
    page_id = page["id"]
    page_title = page["title"]

    # Получите данные содержимого страницы по её ID
    page_data = confluence.get_page_by_id(page_id, expand="body.storage")
    body = page_data["body"]["storage"]["value"]

    # Извлеките нужные данные из содержимого страницы с помощью BeautifulSoup или других методов парсинга
    # Ниже приведен пример для получения текста из всех абзацев страницы
    soup = BeautifulSoup(body, "html.parser")
    paragraphs = soup.find_all("p")
    text_data = "\n".join(paragraph.get_text() for paragraph in paragraphs)

    # Создайте словарь с данными для каждой страницы и добавьте его в список all_data
    page_info = {
        "Page ID": page_id,
        "Page Title": page_title,
        "Page Content": text_data,
    }
    all_data.append(page_info)

# Создайте DataFrame из списка всех данных страниц
df = pd.DataFrame(all_data)

# Сохраните DataFrame в файл CSV
df.to_csv("confluence_pages_data.csv", index=False)

print("Данные всех страниц сохранены в файл confluence_pages_data.csv")


Данные всех страниц сохранены в файл confluence_pages_data.csv


In [11]:
!pip install weasyprint
!pip show pillow

Name: Pillow
Version: 10.0.0
Summary: Python Imaging Library (Fork)
Home-page: https://python-pillow.org
Author: Jeffrey A. Clark (Alex)
Author-email: aclark@aclark.net
License: HPND
Location: /usr/local/lib/python3.10/dist-packages
Requires: 
Required-by: bokeh, dopamine-rl, fastai, imageio, imgaug, matplotlib, scikit-image, torchvision, weasyprint, wordcloud


In [12]:
from atlassian import Confluence
import os
from bs4 import BeautifulSoup
import pandas as pd

# Укажите ваши учетные данные и URL сервера Confluence
user = "mortmafaka@gmail.com"
api_key = "wvDJF36QIKtIsoDLXln186D3"
server = "https://nmtechnologies.atlassian.net"

# Создайте экземпляр класса Confluence для взаимодействия с сервером
confluence = Confluence(url=server, username=user, password=api_key)

# Создайте пустой список, в который будут сохраняться данные каждой страницы
all_data = []

# Рекурсивная функция для обработки всех дочерних страниц
def process_page(page_id):
    # Получите данные содержимого страницы по её ID
    page_data = confluence.get_page_by_id(page_id, expand="body.storage,children.page")
    body = page_data["body"]["storage"]["value"]

    # Проверьте, есть ли содержимое на странице
    if not body:
        print(f"Страница без содержимого: {page_data['title']} (ID: {page_id})")
        return

    # Извлеките ссылки со страницы
    soup = BeautifulSoup(body, "html.parser")
    links = soup.find_all("a", href=True)

    # Создайте словарь с данными для текущей страницы и добавьте его в список all_data
    page_info = {
        "Page ID": page_id,
        "Page Title": page_data["title"],
        "Page Content": body,
    }
    all_data.append(page_info)

    # Рекурсивно обрабатывайте дочерние страницы (если есть ссылки)
    for link in links:
        href = link["href"]
        if "/wiki/pages/viewpage.action?pageId=" in href:
            child_page_id = href.split("pageId=")[1]
            process_page(child_page_id)

# Получите список всех страниц из указанного пространства (space)
all_pages = confluence.get_all_pages_from_space(space="NSJ")

# Переберите список всех страниц и вызовите функцию для обработки каждой страницы
for page in all_pages:
    process_page(page["id"])

# Создайте DataFrame из списка всех данных страниц
df = pd.DataFrame(all_data)

# Сохраните DataFrame в файл CSV
df.to_csv("confluence_pages_data.csv", index=False)

print("Данные всех страниц сохранены в файл confluence_pages_data.csv")


Страница без содержимого: Release notes (ID: 30343170)
Страница без содержимого: Boss (ID: 30638081)
Страница без содержимого: Documentation (ID: 30670849)
Страница без содержимого: Testing (ID: 30736385)
Страница без содержимого: Quotes (ID: 30769153)
Страница без содержимого: Execution (ID: 32800769)
Страница без содержимого: Slippage (ID: 32931841)
Данные всех страниц сохранены в файл confluence_pages_data.csv


In [13]:
import pandas as pd
df = pd.read_csv('confluence_pages_data.csv')

In [14]:
df.head()

,Page ID,Page Title,Page Content
0,30114022,NTGRP SoftDev Java,"<ac:layout><ac:layout-section ac:type=""fixed-w..."
1,30343182,Сессии,<p><em>Сессия (session)</em> задает промежуток...
2,30507039,Выходные ноды,<p>Выходные ноды называются также <strong>кана...
3,30539779,Ноды,<p>boss получает котировки извне (от контраген...
4,30605340,Промежуточные ноды,<p>Промежуточные ноды хранятся в таблице <code...


In [15]:
import pandas as pd

# Загрузите данные из CSV файла в DataFrame
df = pd.read_csv('confluence_pages_data.csv')

# Создайте новый DataFrame с колонками "Page Title" и "Page Content"
new_df = pd.DataFrame(columns=['Page Title', 'Page Content'])

# Добавьте данные из исходного DataFrame в новый DataFrame
for _, row in df.iterrows():
    title = row['Page Title']
    content = row['Page Content']
    new_df = new_df.append({'Page Title': title, 'Page Content': content}, ignore_index=True)

# Сохраните новый DataFrame в файл CSV
new_df.to_csv('confluence_pages_data_new.csv', index=False)

print("Данные сохранены в файл confluence_pages_data_new.csv")


Данные сохранены в файл confluence_pages_data_new.csv


<ipython-input-15-d4a7db989fcb>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append({'Page Title': title, 'Page Content': content}, ignore_index=True)
<ipython-input-15-d4a7db989fcb>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append({'Page Title': title, 'Page Content': content}, ignore_index=True)
<ipython-input-15-d4a7db989fcb>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append({'Page Title': title, 'Page Content': content}, ignore_index=True)
<ipython-input-15-d4a7db989fcb>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append({'Page Title': title, '

asd


In [17]:
import pandas as pd

# Загрузите данные из CSV файла в DataFrame
df = pd.read_csv('confluence_pages_data.csv')

# Откройте файл для записи в текстовый формат
with open('confluence_pages_data.txt', 'w', encoding='utf-8') as file:
    # Итерируйтесь по DataFrame и записывайте каждую строку в файл
    for _, row in df.iterrows():
        title = row['Page Title']
        content = row['Page Content']
        file.write(f"{title}\n{content}\n\n")

print("Данные сохранены в файл confluence_pages_data.txt")


Данные сохранены в файл confluence_pages_data.txt
